Проект 3. О вкусной и здоровой пище

# Start

In [2]:
import warnings; warnings.simplefilter('ignore')

import random
import collections

import pandas as pd
import numpy as np
import os
import re

import datetime
from datetime import datetime, timedelta

import json  
from pprint import pprint
import xml.etree.ElementTree as ET

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


%matplotlib inline

## function

In [18]:
def column_content_analysis(column_name_list, column_type='str', more_than = 10):
    '''
        Функция для определения типовых параметров данных, а также определние "замусоренности".
        На вход принимает список столбцов датафрейма
    '''
    # получаем характеристики
    row_count, col_count = ta.shape

    # проходим циклом по списку колонок    
    for column_name in column_name_list:
        print('-'*60)
        print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
        print('-'*60)
        if column_name not in ta.columns:
            print('|  Внимание!!! Указанная колонка не найдена в датасете. \n')
            break

        col_count = ta[column_name].count()
        col_type = ta[column_name].dtype
        print('|  Тип данных:', col_type) 
        print('|  Заполнено значений:',  col_count, 'из', row_count)
        print('|  Отсутсвующие значения:', row_count - col_count)
        print('|  Полнота данных: ', round(col_count / row_count * 100 ,2), '%', sep='')
        print('|  Количество уникальных значений:', ta[column_name].nunique())

        print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
              (ta[column_name].value_counts() > more_than).sum())

        if col_type == 'object':
            # Количество пробелов в начале в конце строки
            print('|  Количество раз пробелы найдены:')
            find_list = [len(re.findall(r'^\s+\w?', str(x))) for x in ta[column_name]]
            print('|    в начале строки:', sum(find_list))
            
            find_list = [len(re.findall(r'\w?\s+$', str(x))) for x in ta[column_name]]
            print('|    в конце  строки:', sum(find_list))
            
            find_list = [len(re.findall(r'[^\s\d\w]', str(x))) for x in ta[column_name]]
            print('|  Количество найденных специальных символов:', sum(find_list))
        
        if col_type in ['int', 'float']:
            print('|  Медиана: ', ta[column_name].median())
            
            perc25 = ta[column_name].describe().loc['25%']
            perc75 = ta[column_name].describe().loc['75%']
            iqr = perc75 - perc25
            range_left = perc25 - 1.5 * iqr
            range_right = perc75 + 1.5 * iqr
            
            # Границы выбросов
            print('|  IQR: ', iqr, '; границы выбросов: [', round(range_left, 2), '; ', round(range_right,2),']' , sep='')
            out_of_range_left = ta[column_name].loc[ta[column_name] < range_left]
            out_of_range_right = ta[column_name].loc[ta[column_name] > range_right]
            out_of_range = len(out_of_range_left) + len(out_of_range_right)
            if out_of_range > 0:
                print('|  Данные СОДЕРЖАТ выбросы, количество значений: ', out_of_range)
                if len(out_of_range_left) > 0:
                    print('|    Слева  от IQR (уникальные):', out_of_range_left.unique())
                if len(out_of_range_right) > 0:
                    print('|    Справа от IQR (уникальные):', out_of_range_right.unique())
         
        print('|---\n|  Все уникальные значения:', ta[column_name].unique() , '\n')
        

    # end function


In [8]:
project_dir = r'/content/drive/My Drive/Colab Notebooks/data/u3/p3/'
os.listdir(project_dir)

['main_task.csv']

## description

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

`Restaurant_id` — идентификационный номер ресторана / сети ресторанов;

`City` — город, в котором находится ресторан;

`Cuisine Style` — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;

`Ranking` — место, которое занимает данный ресторан среди всех ресторанов своего города;

`Rating` — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);

`Price Range` — диапазон цен в ресторане;

`Number of Reviews` — количество отзывов о ресторане;

`Reviews` — данные о двух отзывах, которые отображаются на сайте ресторана;

`URL_TA` — URL страницы ресторана на TripAdvosor;

`ID_TA` — идентификатор ресторана в базе данных TripAdvisor.

In [ ]:
ta = pd.read_csv(project_dir + 'main_task.csv')


In [20]:
# приводим все названия колонок к нижнему регистру
ta.columns = [str.lower(x) for x in ta.columns]

# получаем список всех колонок
all_columns = ta.columns

In [26]:
column_content_analysis(ta.columns)

------------------------------------------------------------
|	 Отчет по колонке [ restaurant_id ] 					
------------------------------------------------------------
|  Тип данных: object
|  Заполнено значений: 40000 из 40000
|  Отсутсвующие значения: 0
|  Полнота данных: 100.0%
|  Количество уникальных значений: 11909
|  Значений, встретившихся в столбце более 10 раз: 640
|  Количество раз пробелы найдены:
|    в начале строки: 0
|    в конце  строки: 0
|  Количество найденных специальных символов: 0
|---
|  Все уникальные значения: ['id_5569' 'id_1535' 'id_352' ... 'id_7068' 'id_8887' 'id_6057'] 

------------------------------------------------------------
|	 Отчет по колонке [ city ] 					
------------------------------------------------------------
|  Тип данных: object
|  Заполнено значений: 40000 из 40000
|  Отсутсвующие значения: 0
|  Полнота данных: 100.0%
|  Количество уникальных значений: 31
|  Значений, встретившихся в столбце более 10 раз: 31
|  Количество раз пробелы найд

In [25]:
ta.sample(29)

,restaurant_id,city,cuisine style,ranking,rating,price range,number of reviews,reviews,url_ta,id_ta
13351,id_7937,Paris,NaN,7938.0,3.5,NaN,24.0,"[['18€ for a cup and a cone of ice cream', 'Un...",/Restaurant_Review-g187147-d12500076-Reviews-G...,d12500076
37498,id_2296,Vienna,['Pub'],2298.0,4.5,NaN,5.0,"[['Impressing cosy place, openminded people', ...",/Restaurant_Review-g190454-d8757608-Reviews-Da...,d8757608
24663,id_1069,Hamburg,NaN,1071.0,4.5,NaN,5.0,"[[], []]",/Restaurant_Review-g187331-d10831212-Reviews-B...,d10831212
27258,id_2351,Rome,"['Fast Food', 'Mediterranean', 'Middle Eastern...",2352.0,4.0,$,241.0,"[['Dinner near the hotel', 'Best Shawarma in R...",/Restaurant_Review-g187791-d794370-Reviews-Sha...,d794370
31546,id_13460,London,NaN,13470.0,4.0,NaN,4.0,[['Overpriced with small portions and unfrien....,/Restaurant_Review-g186338-d10750344-Reviews-C...,d10750344
8561,id_1477,Lisbon,"['European', 'Portuguese']",1478.0,4.0,$$ - $$$,31.0,"[['Hidden gem', 'Very nice food and staff with...",/Restaurant_Review-g189158-d10350218-Reviews-T...,d10350218
2636,id_734,Zurich,"['European', 'Mediterranean', 'Delicatessen', ...",738.0,4.0,$$ - $$$,71.0,"[['Fantastic!', 'Great evening'], ['01/12/2017...",/Restaurant_Review-g188113-d809849-Reviews-Jda...,d809849
14374,id_877,Vienna,"['Mediterranean', 'Barbecue', 'Turkish', 'Midd...",878.0,4.0,$$ - $$$,171.0,"[['Good Turkish food at reasonable price', 'Ve...",/Restaurant_Review-g190454-d799627-Reviews-Ken...,d799627
27103,id_5506,Madrid,NaN,5509.0,4.0,NaN,14.0,[['Good and confortable eating at an outstand....,/Restaurant_Review-g187514-d717344-Reviews-Met...,d717344
4269,id_258,Amsterdam,"['Mediterranean', 'Spanish', 'Vegetarian Frien...",259.0,4.5,$$ - $$$,342.0,[['Perfect- best place we’ve been in Amsterda....,/Restaurant_Review-g188590-d2297611-Reviews-Or...,d2297611


In [23]:
ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price range        26114 non-null  object 
 6   number of reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [ ]:
ta.sample(9)

# END